<a href="https://colab.research.google.com/github/Edsonalex/class-planner/blob/main/Plano_de_Aula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install google-genai
!pip install -q pdfkit
!apt-get update -qq
!apt-get install -q wkhtmltopdf

In [115]:
# Configura a API Key do Google Gemini

import os
import google.generativeai as genai
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [116]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [117]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [118]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [119]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [120]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [121]:
##########################################
# --- Agente 1: Planejador Pedagógico --- #
##########################################
class PlanejadorPedagogico:
    def __init__(self, model):
        self.agente_conteudo = AgenteConteudo(model)
        self.agente_atividades = AgenteAtividades(model)
        self.agente_avaliacao = AgenteAvaliacao(model)
        self.agente_agenda = AgenteAgenda(model)

    def planejar_aula(self, serie, disciplina, tema, duracao):
        conteudo = self.agente_conteudo.sugerir_conteudo(serie, disciplina, tema)
        atividades = self.agente_atividades.sugerir_atividades(serie, tema)
        avaliacoes = self.agente_avaliacao.sugerir_avaliacoes(tema)
        cronograma = self.agente_agenda.gerar_cronograma(duracao, conteudo, atividades, avaliacoes)

        return {
            "Série": serie,
            "Disciplina": disciplina,
            "Tema": tema,
            "Conteúdo": conteudo,
            "Atividades": atividades,
            "Avaliações": avaliacoes,
            "Cronograma": cronograma
        }

In [136]:
#########################################
# --- Agente 2: Criador de Conteúdo --- #
#########################################
class AgenteConteudo:
    def __init__(self, model):
        self.model = model
    def sugerir_conteudo(self, serie, disciplina, tema):
        prompt = (
            f"Você é um especialista em educação básica. Sugira um conteúdo didático para uma aula de {disciplina} "
            f"do {serie} ano do ensino fundamental sobre o tema '{tema}', de acordo com a BNCC. Seja objetivo e sucinto. Escreva como um professor."
        )
        response = self.model.generate_content(prompt)
        return response.text

In [123]:
###########################################
# --- Agente 3: Criador de Atividades --- #
###########################################
class AgenteAtividades:
    def __init__(self, model):
        self.model = model
    def sugerir_atividades(self, serie, tema):
        prompt = (
            f"Sugira duas atividades lúdicas e práticas para alunos do {serie} ano do ensino fundamental, "
            f"sobre o tema '{tema}'. As atividades devem ser simples e possíveis de realizar em sala de aula."
        )
        response = self.model.generate_content(prompt)
        return response.text.split("\n")

In [124]:
###########################################
# --- Agente 4: Criador de Avaliações --- #
###########################################
class AgenteAvaliacao:
    def __init__(self, model):
        self.model = model
    def sugerir_avaliacoes(self, tema):
        prompt = (
            f"Sugira instrumentos de avaliação divididas nas cateorias Formativa (autoavaliação e observação) e Somativa(provas, testes, trabalhos) para o tema {tema} e a série {serie}.",
            f"Recomende avaliação escrita com problemas contextualizados."
        )
        response = self.model.generate_content(prompt)
        return response.text.split("\n")

In [125]:
##########################################
# --- Agente 5: Criador de Agenda --- #
##########################################
class AgenteAgenda:
    def __init__(self, model):
      self.model = model
    def gerar_cronograma(self, duracao, conteudo, atividades, avaliacoes):
        prompt = (
            f"Organize os conteúdos, atividades e avaliações num cronograma semanal em formato de calendário considerando o tema{tema}, a duração {duracao}, as atividades propostas {atividades} e as avalaliações {avaliacoes}.",
            f"Considere número de 4 aulas por semana e o tempo de 50 minutos necessário por aula"
        )
        response = self.model.generate_content(prompt)
        return response.text.split("\n")

In [ ]:
##############################
# --- Prompt e Resultado --- #
##############################
def iniciar_prompt():
    print("=== Gerador de Plano de Aula ===")
    serie = input("Série (ex: 5º): ")
    disciplina = input("Disciplina (ex: Matemática): ")
    tema = input("Tema (ex: Frações): ")
    duracao = int(input("Duração em semanas: "))
    return serie, disciplina, tema, duracao

import google.generativeai as genai

model = genai.GenerativeModel(MODEL_ID)
planejador = PlanejadorPedagogico(model)

serie, disciplina, tema, duracao = iniciar_prompt()

plano = planejador.planejar_aula(serie, disciplina, tema, duracao)

print("\n\n🚀 Iniciando o Sistema de Criação de Planejamento Pedagógico 🚀")
print(f"Maravilha! Vamos então criar o planejamento em {disciplina} para alunos da {serie} série com o tema {tema} e com duração de {duracao} semana(s).")

conteudo = plano["Conteúdo"]
print("\n--- 📝 Resultado do Gerador de Conteúdo (Conteúdo) ---\n")
display(to_markdown(conteudo))
print("--------------------------------------------------------------")

atividades = plano["Atividades"]
print("\n--- 📝 Resultado do Gerador de Atividades (Atividades) ---\n")
for atividade in atividades:
    display(to_markdown(atividade))
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")

avaliacoes = plano["Avaliações"]
print("\n--- 📝 Resultado do Gerador de Avaliações (Avaliações) ---\n")
for avaliacao in avaliacoes:
    display(to_markdown(avaliacao))
print("--------------------------------------------------------------")

cronograma = plano["Cronograma"]
print("\n--- 📝 Resultado do Gerador de Cronograma (Cronograma) ---\n")
if isinstance(cronograma, list):
    for item in cronograma:
        display(to_markdown(item))
elif isinstance(cronograma, dict):
    for key, value in cronograma.items():
        display(to_markdown(f"{key}:\n{value}"))
else:
    display(to_markdown(str(cronograma)))

print("--------------------------------------------------------------")